In [1]:
!pip install captum
%pip install .

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Processing /Users/harryamad/Google Drive/encoder_attribution_priors
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for lfxai: filename=lfxai-0.1.1-py3-none-any.whl size=30798 sha256=e0b936070c02a9491fa6dc707243322a2daf485a8b58b9a6209c704930cf9b09
  Stored in directory: /private/var/folders/v1/8lvgykdd7jz1mgwv6lgglt540000gn/T/pip-ephem-wheel-cache-o4hskndo/wheels/d8/ad/fa/7f23a7351b78cbce54840af97b785c69f88ecbff5d524953e6
Successfully built lfxai
  Attempting uninstall: lfxai
    Found existing installation: lfxai 0.1.1
    Unins

In [2]:
import argparse
import csv
import itertools
import logging
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torchvision
from captum.attr import GradientShap, IntegratedGradients, Saliency
from scipy.stats import spearmanr
from torch.utils.data import DataLoader, RandomSampler, Subset
from torchvision import transforms

from lfxai.explanations.examples import (
    InfluenceFunctions,
    NearestNeighbours,
    SimplEx,
    TracIn,
)
from lfxai.explanations.features import attribute_auxiliary, attribute_individual_dim, tensor_attribution, attribute_training
from lfxai.models.images import (
    VAE,
    AutoEncoderMnist,
    ClassifierMnist,
    DecoderBurgess,
    DecoderMnist,
    EncoderBurgess,
    EncoderMnist,
)
from lfxai.models.losses import BetaHLoss, BtcvaeLoss, EntropyLoss, PearsonLoss
from lfxai.models.pretext import Identity, Mask, RandomNoise
from lfxai.utils.datasets import MaskedMNIST
from lfxai.utils.feature_attribution import generate_masks
from lfxai.utils.metrics import (
    compute_metrics,
    cos_saliency,
    count_activated_neurons,
    entropy_saliency_tensor,
    pearson_saliency,
    similarity_rates,
    spearman_saliency,
    pearson_saliency_tensor
)
from lfxai.utils.visualize import (
    correlation_latex_table,
    plot_pretext_saliencies,
    plot_pretext_top_example,
    plot_vae_saliencies,
    vae_box_plots,
)

In [18]:
a = torch.tensor([[-11],[2]])
b = torch.tensor([[3],[4]])

In [16]:
torch.cat([a,b], axis=1)

tensor([[1, 3],
        [2, 4]])

In [19]:
torch.abs(a)

tensor([[11],
        [ 2]])

In [69]:
latents = []
latents.append(torch.tensor([[1,2,3], [4,5,6]]).detach().cpu().numpy())
latents = np.concatenate(latents)
latents = np.expand_dims(latents, (2, 3))
latents

array([[[[1]],

        [[2]],

        [[3]]],


       [[[4]],

        [[5]],

        [[6]]]])

In [71]:
latents = []
latents.append(torch.tensor([[1,2,3], [4,5,6]]))
latents = torch.cat(latents)
latents = torch.unsqueeze(torch.unsqueeze(latents, 2), 2)
latents

tensor([[[[1]],

         [[2]],

         [[3]]],


        [[[4]],

         [[5]],

         [[6]]]])

In [59]:
def test_attribution(
    encoder: callable,
    dim_latent: int,
    data_loader: torch.utils.data.DataLoader,
    device: torch.device,
    attr_method,
    baseline: torch.Tensor,
):

    attributions = []
    latents = []
    for input_batch, _ in data_loader:
        input_batch = input_batch.to(device)
        attributions_batch = []
        latents.append(encoder(input_batch))
        for dim in range(dim_latent):
            attribution = (attr_method.attribute(input_batch, baseline, target=dim))
            attributions_batch.append(attribution)
        attributions.append(torch.cat(attributions_batch, axis=1))
    
    latents = torch.cat(latents)
    attributions = torch.cat(attributions)
    attributions = torch.abs(torch.unsqueeze(torch.unsqueeze(latents, 2), 2) * attributions)
    return attributions

In [3]:
W=32
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
data_dir = Path.cwd() / "data/mnist"

img_size = (1, W, W)
encoder = EncoderBurgess(img_size, 3)

baseline_image = torch.zeros((1, 1, W, W), device=device)
test_dataset = torchvision.datasets.MNIST(data_dir, train=False, download=True)


test_transform = transforms.Compose([transforms.Resize(W), transforms.ToTensor()])
test_dataset.transform = test_transform
test_dataset.data, test_dataset.targets = test_dataset.data[[1,2,3,4,5]], test_dataset.targets[[1,2,3,4,5]]
test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=False)

gradshap = GradientShap(encoder.mu)

In [55]:
saliency_tensor = tensor_attribution(encoder.mu, 3, test_loader, device, gradshap, baseline_image) 
saliency_array = attribute_individual_dim(encoder.mu, 3, test_loader, device, gradshap, baseline_image)

In [70]:
def off_diagonal_sum(mat: np.ndarray) -> np.ndarray:
    """
    Computes the sum of of-diagonal matrix elements
    Args:
        mat: matrix

    Returns:
        sum of the off diagonal elements of mat
    """
    return np.sum(mat) - np.trace(mat)
def pearson_saliency(saliency: np.ndarray) -> np.ndarray:
    """
    Computes the average Pearson correlation between different saliency maps
    Args:
        saliency: saliency maps stacked together (indexed by the first tensor dimension)

    Returns:
        Pearson correlation between saliency maps
    """
    latent_dim = saliency.shape[1]
    corr = np.corrcoef(saliency.swapaxes(0, 1).reshape(latent_dim, -1))

    print(corr)
    return off_diagonal_sum(corr) / (latent_dim * (latent_dim - 1))

def pearson_saliency_tensor(saliency):
    """
    Computes the average Pearson correlation between different saliency maps with gradients
    Args:
        saliency: saliency maps stacked together (indexed by the first tensor dimension)

    Returns:
        Pearson correlation between saliency maps
    """
    latent_dim = saliency.shape[1]
    corr = torch.corrcoef(torch.reshape(torch.swapaxes(saliency, 0, 1), (latent_dim, -1)))
    print(torch.reshape(torch.swapaxes(saliency, 0, 1), (latent_dim, -1)))
    print(corr)

    return (torch.sum(corr) - torch.trace(corr)) / (latent_dim * (latent_dim - 1))


In [68]:
pearson_saliency(saliency_array)

[[1.         0.60455571 0.6439361 ]
 [0.60455571 1.         0.62773663]
 [0.6439361  0.62773663 1.        ]]


0.6254094780191829

In [69]:
pearson_saliency_tensor(saliency_tensor)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<UnsafeViewBackward0>)
tensor([[1.0000, 0.5811, 0.5977],
        [0.5811, 1.0000, 0.6213],
        [0.5977, 0.6213, 1.0000]], grad_fn=<ClampBackward1>)


tensor(0.6001, grad_fn=<DivBackward0>)

In [58]:
t = torch.tensor([[1.,1.,1.,1.,1.,1.,1.,1.], [1.,1.,1.,1.,1.,1.,1.,1.], [1.,1.,1.,1.,1.,1.,1.,1.]])
t

tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [59]:
np.corrcoef(t)

array([[nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan]])

In [60]:
torch.corrcoef(t)

tensor([[nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan]])

In [115]:
from scipy.stats import entropy
def entropy_saliency(saliency: np.ndarray) -> np.ndarray:
    """
    Computes the entropy of different saliency maps
    Args:
        saliency: saliency maps stacked together (indexed by the first tensor dimension)

    Returns:
        Entropy between saliency maps
    """
    latent_dim = saliency.shape[1]
    saliency_reshaped = np.reshape(saliency.swapaxes(1, -1), (-1, latent_dim))
    salient_pixels = saliency_reshaped.sum(1) > 0

    saliency_filtered = saliency_reshaped[salient_pixels]

    entropy_ar = entropy(saliency_filtered, axis=1)

    return np.mean(entropy_ar)

In [116]:
from torch.distributions import Categorical
def entropy_saliency_tensor(saliency: np.ndarray) -> np.ndarray:
    """
    Computes the entropy of different saliency maps
    Args:
        saliency: saliency maps stacked together (indexed by the first tensor dimension)

    Returns:
        Entropy between saliency maps
    """
    latent_dim = saliency.shape[1]
    saliency_reshaped = torch.reshape(torch.swapaxes(saliency, 1, -1), (-1, latent_dim))
    salient_pixels = saliency_reshaped.sum(1) > 0

    saliency_filtered = saliency_reshaped[salient_pixels]

    entropy_ar = Categorical(probs = saliency_filtered).entropy()

    #entropy_ar = entropy(saliency_filtered, axis=1)
    return torch.mean(entropy_ar)

In [117]:
entropy_saliency(saliency_array)

1.0986122

In [118]:
entropy_saliency_tensor(saliency_tensor)

tensor(1.0986, grad_fn=<MeanBackward0>)

In [ ]:
def disvae_feature_importance(
    random_seed: int = 1,
    batch_size: int = 300,
    n_plots: int = 20,
    #n_runs: int = 5,
    n_runs: int = 2,
    dim_latent: int = 3,
    #n_epochs: int = 100,
    n_epochs: int = 5,
    #beta_list: list = [1, 5, 10],
    beta_list: list = [1],
) -> None:
    # Initialize seed and device
    np.random.seed(random_seed)
    torch.random.manual_seed(random_seed)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    # Load MNIST
    W = 32
    img_size = (1, W, W)
    data_dir = Path.cwd() / "drive/MyDrive/encoder_attribution_priors/experiments/data/mnist"
    train_dataset = torchvision.datasets.MNIST(data_dir, train=True, download=True)
    test_dataset = torchvision.datasets.MNIST(data_dir, train=False, download=True)
    train_transform = transforms.Compose([transforms.Resize(W), transforms.ToTensor()])
    test_transform = transforms.Compose([transforms.Resize(W), transforms.ToTensor()])
    train_dataset.transform = train_transform
    test_dataset.transform = test_transform
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False
    )

    # Create saving directory
    save_dir = Path.cwd() / "drive/MyDrive/encoder_attribution_priors/experiments/results/mnist/vae"
    if not save_dir.exists():

        print(f"Creating saving directory {save_dir}")
        os.makedirs(save_dir)

    # Define the computed metrics and create a csv file with appropriate headers
    loss_list = [EntropyLoss(alpha=100), BetaHLoss(), BtcvaeLoss(is_mss=False, n_data=len(train_dataset))]
    metric_list = [
        pearson_saliency,
        spearman_saliency,
        cos_saliency,
        entropy_saliency,
        count_activated_neurons,
    ]
    metric_names = [
        "Pearson Correlation",
        "Spearman Correlation",
        "Cosine",
        "Entropy",
        "Active Neurons",
    ]
    headers = ["Loss Type", "Beta"] + metric_names
    csv_path = save_dir / "metrics.csv"
    if not csv_path.is_file():
        print(f"Creating metrics csv in {csv_path}")

        with open(csv_path, "w") as csv_file:
            dw = csv.DictWriter(csv_file, delimiter=",", fieldnames=headers)
            dw.writeheader()

    for beta, loss, run in itertools.product(
        beta_list, loss_list, range(1, n_runs + 1)
    ):
        # Initialize vaes
        encoder = EncoderBurgess(img_size, dim_latent)
        decoder = DecoderBurgess(img_size, dim_latent)
        loss.beta = beta
        name = f"{str(loss)}-vae_beta{beta}_run{run}"
        model = VAE(img_size, encoder, decoder, dim_latent, loss, name=name)
        print(f"Now fitting {name}")

        model.fit(device, train_loader, test_loader, save_dir, n_epochs)
        model.load_state_dict(torch.load(save_dir / (name + ".pt")), strict=False)

        # Compute test-set saliency and associated metrics
        baseline_image = torch.zeros((1, 1, W, W), device=device)
        gradshap = GradientShap(encoder.mu)
        attributions = attribute_individual_dim(
            encoder.mu, dim_latent, test_loader, device, gradshap, baseline_image
        )
        metrics = compute_metrics(attributions, metric_list)
        results_str = "\t".join(
            [f"{metric_names[k]} {metrics[k]:.2g}" for k in range(len(metric_list))]
        )
        print(f"Model {name} \t {results_str}")


        # Save the metrics
        with open(csv_path, "a", newline="") as csv_file:
            writer = csv.writer(csv_file, delimiter=",")
            writer.writerow([str(loss), beta] + metrics)

        # Plot a couple of examples
        plot_idx = [
            torch.nonzero(test_dataset.targets == (n % 10))[n // 10].item()
            for n in range(n_plots)
        ]
        images_to_plot = [test_dataset[i][0].numpy().reshape(W, W) for i in plot_idx]
        fig = plot_vae_saliencies(images_to_plot, attributions[plot_idx])
        fig.savefig(save_dir / f"{name}.pdf")
        plt.close(fig)

    fig = vae_box_plots(pd.read_csv(csv_path), metric_names)
    fig.savefig(save_dir / "metric_box_plots.pdf")
    plt.close(fig)

In [ ]:
disvae_feature_importance()

Now fitting Entropy-vae_beta1_run1


Model Entropy-vae_beta1_run1 	 Pearson Correlation 0.52	Spearman Correlation 0.99	Cosine 0.82	Entropy 0.066	Active Neurons 1
Now fitting Entropy-vae_beta1_run2


Model Entropy-vae_beta1_run2 	 Pearson Correlation 0.38	Spearman Correlation 0.99	Cosine 0.73	Entropy 0.22	Active Neurons 1
Now fitting Beta-vae_beta1_run1


Model Beta-vae_beta1_run1 	 Pearson Correlation 0.42	Spearman Correlation 0.99	Cosine 0.73	Entropy 0.45	Active Neurons 1.3
Now fitting Beta-vae_beta1_run2


Model Beta-vae_beta1_run2 	 Pearson Correlation 0.33	Spearman Correlation 0.98	Cosine 0.62	Entropy 0.74	Active Neurons 1.3
Now fitting TC-vae_beta1_run1


Model TC-vae_beta1_run1 	 Pearson Correlation 0.31	Spearman Correlation 0.99	Cosine 0.63	Entropy 0.74	Active Neurons 1.3
Now fitting TC-vae_beta1_run2


Model TC-vae_beta1_run2 	 Pearson Correlation 0.31	Spearman Correlation 0.98	Cosine 0.61	Entropy 0.72	Active Neurons 1.3


In [3]:
def entropy_vae(
    random_seed: int = 1,
    batch_size: int = 300,
    n_plots: int = 20,
    n_runs: int = 1,
    dim_latent: int = 3,
    n_epochs: int = 5,
    alpha_list: list = [50],
) -> None:
    # Initialize seed and device
    np.random.seed(random_seed)
    torch.random.manual_seed(random_seed)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    # Load MNIST
    W = 32
    img_size = (1, W, W)
    data_dir = Path.cwd() / "drive/MyDrive/encoder_attribution_priors/experiments/data/mnist"
    train_dataset = torchvision.datasets.MNIST(data_dir, train=True, download=True)
    #train_dataset.data, train_dataset.targets = train_dataset.data[[1,2,3,4,5]], train_dataset.targets[[1,2,3,4,5]]
    test_dataset = torchvision.datasets.MNIST(data_dir, train=False, download=True)
    #test_dataset.data, test_dataset.targets = test_dataset.data[[1,2,3,4,5]], test_dataset.targets[[1,2,3,4,5]]
    train_transform = transforms.Compose([transforms.Resize(W), transforms.ToTensor()])
    test_transform = transforms.Compose([transforms.Resize(W), transforms.ToTensor()])
    train_dataset.transform = train_transform
    test_dataset.transform = test_transform
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False
    )

    # Create saving directory
    save_dir = Path.cwd() / "drive/MyDrive/encoder_attribution_priors/experiments/results/mnist/entropy_vae"
    if not save_dir.exists():

        print(f"Creating saving directory {save_dir}")
        os.makedirs(save_dir)

    # Define the computed metrics and create a csv file with appropriate headers
    loss_list = [EntropyLoss()]
    metric_list = [
        pearson_saliency,
        entropy_saliency,
        count_activated_neurons,
    ]
    metric_names = [
        "Pearson Correlation",
        "Entropy",
        "Active Neurons",
    ]
    headers = ["Loss Type", "Alpha"] + metric_names
    csv_path = save_dir / "metrics.csv"
    if not csv_path.is_file():
        print(f"Creating metrics csv in {csv_path}")

        with open(csv_path, "w") as csv_file:
            dw = csv.DictWriter(csv_file, delimiter=",", fieldnames=headers)
            dw.writeheader()

    for alpha, loss, run in itertools.product(
        alpha_list, loss_list, range(1, n_runs + 1)
    ):
        # Initialize vaes
        encoder = EncoderBurgess(img_size, dim_latent)
        decoder = DecoderBurgess(img_size, dim_latent)
        loss.alpha = alpha
        name = f"{str(loss)}-vae_alpha{alpha}_run{run}"
        model = VAE(img_size, encoder, decoder, dim_latent, loss, name=name)
        print(f"Now fitting {name}")

        model.fit(device, train_loader, test_loader, save_dir, n_epochs)
        model.load_state_dict(torch.load(save_dir / (name + ".pt")), strict=False)

        # Compute test-set saliency and associated metrics
        baseline_image = torch.zeros((1, 1, W, W), device=device)
        gradshap = GradientShap(encoder.mu)
        attributions = attribute_individual_dim(
            encoder.mu, dim_latent, test_loader, device, gradshap, baseline_image
        )
        metrics = compute_metrics(attributions, metric_list)
        results_str = "\t".join(
            [f"{metric_names[k]} {metrics[k]:.2g}" for k in range(len(metric_list))]
        )
        print(f"Model {name} \t {results_str}")


        # Save the metrics
        with open(csv_path, "a", newline="") as csv_file:
            writer = csv.writer(csv_file, delimiter=",")
            writer.writerow([str(loss), alpha] + metrics)

        # Plot a couple of examples
        plot_idx = [
            torch.nonzero(test_dataset.targets == (n % 10))[n // 10].item()
            for n in range(n_plots)
        ]
        images_to_plot = [test_dataset[i][0].numpy().reshape(W, W) for i in plot_idx]
        fig = plot_vae_saliencies(images_to_plot, attributions[plot_idx])
        fig.savefig(save_dir / f"{name}.pdf")
        plt.close(fig)

    fig = vae_box_plots(pd.read_csv(csv_path), metric_names, loss='entropy')
    fig.savefig(save_dir / "metric_box_plots.pdf")
    plt.close(fig)

In [4]:
entropy_vae()

  0%|          | 0/200 [00:00<?, ?batches/s]

Now fitting Entropy-vae_alpha50_run1


KeyboardInterrupt: 

In [3]:
def pearson_vae(
    random_seed: int = 1,
    batch_size: int = 300,
    n_plots: int = 20,
    n_runs: int = 1,
    dim_latent: int = 3,
    n_epochs: int = 10,
    alpha_list: list = [1],
) -> None:
    # Initialize seed and device
    np.random.seed(random_seed)
    torch.random.manual_seed(random_seed)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    # Load MNIST
    W = 32
    img_size = (1, W, W)
    data_dir = Path.cwd() / "drive/MyDrive/encoder_attribution_priors/experiments/data/mnist"
    train_dataset = torchvision.datasets.MNIST(data_dir, train=True, download=True)
    test_dataset = torchvision.datasets.MNIST(data_dir, train=False, download=True)
    train_transform = transforms.Compose([transforms.Resize(W), transforms.ToTensor()])
    test_transform = transforms.Compose([transforms.Resize(W), transforms.ToTensor()])
    train_dataset.transform = train_transform
    test_dataset.transform = test_transform
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False
    )

    # Create saving directory
    save_dir = Path.cwd() / "drive/MyDrive/encoder_attribution_priors/experiments/results/mnist/pearson_vae"
    if not save_dir.exists():

        print(f"Creating saving directory {save_dir}")
        os.makedirs(save_dir)

    # Define the computed metrics and create a csv file with appropriate headers
    loss_list = [PearsonLoss()]
    metric_list = [
        pearson_saliency,
        entropy_saliency,
        count_activated_neurons,
    ]
    metric_names = [
        "Pearson Correlation",
        "Entropy",
        "Active Neurons",
    ]
    headers = ["Loss Type", "Alpha"] + metric_names
    csv_path = save_dir / "metrics.csv"
    if not csv_path.is_file():
        print(f"Creating metrics csv in {csv_path}")

        with open(csv_path, "w") as csv_file:
            dw = csv.DictWriter(csv_file, delimiter=",", fieldnames=headers)
            dw.writeheader()

    for alpha, loss, run in itertools.product(
        alpha_list, loss_list, range(1, n_runs + 1)
    ):
        # Initialize vaes
        encoder = EncoderBurgess(img_size, dim_latent)
        decoder = DecoderBurgess(img_size, dim_latent)
        loss.alpha = alpha
        name = f"{str(loss)}-vae_alpha{alpha}_run{run}"
        model = VAE(img_size, encoder, decoder, dim_latent, loss, name=name)
        print(f"Now fitting {name}")

        model.fit(device, train_loader, test_loader, save_dir, n_epochs)
        model.load_state_dict(torch.load(save_dir / (name + ".pt")), strict=False)

        # Compute test-set saliency and associated metrics
        baseline_image = torch.zeros((1, 1, W, W), device=device)
        gradshap = GradientShap(encoder.mu)
        attributions = attribute_individual_dim(
            encoder.mu, dim_latent, test_loader, device, gradshap, baseline_image
        )
        metrics = compute_metrics(attributions, metric_list)
        results_str = "\t".join(
            [f"{metric_names[k]} {metrics[k]:.2g}" for k in range(len(metric_list))]
        )
        print(f"Model {name} \t {results_str}")


        # Save the metrics
        with open(csv_path, "a", newline="") as csv_file:
            writer = csv.writer(csv_file, delimiter=",")
            writer.writerow([str(loss), alpha] + metrics)

        # Plot a couple of examples
        plot_idx = [
            torch.nonzero(test_dataset.targets == (n % 10))[n // 10].item()
            for n in range(n_plots)
        ]
        images_to_plot = [test_dataset[i][0].numpy().reshape(W, W) for i in plot_idx]
        fig = plot_vae_saliencies(images_to_plot, attributions[plot_idx])
        fig.savefig(save_dir / f"{name}.pdf")
        plt.close(fig)

    fig = vae_box_plots(pd.read_csv(csv_path), metric_names, loss='entropy')
    fig.savefig(save_dir / "metric_box_plots.pdf")
    plt.close(fig)

In [4]:
pearson_vae()

Now fitting Pearson-vae_alpha1_run1


RuntimeError: ignored

In [6]:
torch.tensor(10.0, requires_grad=True)

tensor(10., requires_grad=True)